# Demo for styringsmøte den 10.03.2025

NB! må kjøre konverting av filer seksjonen først

## Importering av pakke

In [19]:
import os
import pyarrow.parquet as pq
import geopandas as gpd
import pandas as pd
import datetime
import glob
import json
import time as time_module
from shapely.geometry import box
from IPython.display import display
import folium
import re


## Konstante variabler

In [13]:
ROTMAPPE = "Data"
PARTISJON_MAPPE_HOUR = f"{ROTMAPPE}/partitioned_hour"
PARTISJON_MAPPE_MIN = f"{ROTMAPPE}/partitioned_minutes"

## Konverting av filer

In [3]:
def er_geoparquet(filsti):
    """Sjekker om en parquet-fil er en GeoParquet-fil"""
    try:
        # Les metadata fra parquet-filen
        metadata = pq.read_metadata(filsti)
        
        # Sjekk om metadata inneholder noen geometri-kolonner
        schema = pq.read_schema(filsti)
        for navn in schema.names:
            felt = schema.field(navn)
            # Sjekk metadata for kolonnen
            if felt.metadata and ('geo' in felt.metadata or b'geo' in felt.metadata):
                return True
                
        # Les filen med geopandas og se om den inneholder en geometrikolonne
        try:
            gdf = gpd.read_parquet(filsti)
            if hasattr(gdf, 'geometry') and gdf.geometry.name in gdf.columns:
                return True
        except Exception:
            pass
            
        return False
    except Exception:
        return False

In [4]:
def konverter_til_geoparquet(kildefilsti, målmappesti=None):
    """Konverterer ulike geografiske formater til GeoParquet-format"""
    try:
        # Få filnavn uten endelse
        base_navn = os.path.basename(kildefilsti)
        base_navn_uten_endelse = os.path.splitext(base_navn)[0]
        filendelse = os.path.splitext(kildefilsti)[1].lower()
        
        # Bestem målfilsti
        if målmappesti is None:
            # Hvis ingen målmappe er angitt, bruk samme mappe med _geo.parquet
            dir_navn = os.path.dirname(kildefilsti)
            målfilsti = os.path.join(dir_navn, f"{base_navn_uten_endelse}_geo.parquet")
        else:
            # Hvis målmappe er angitt, plasser filen der
            målfilsti = os.path.join(målmappesti, f"{base_navn_uten_endelse}.parquet")
        
        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(målfilsti), exist_ok=True)
        
        gdf = None
        
        # Behandle filen basert på filtype
        if filendelse == '.parquet':
            # Sjekk først om filen er en vanlig parquet-fil
            df = pd.read_parquet(kildefilsti)
            
            # Sjekk etter lat/long kolonner
            if 'longitude' in df.columns and 'latitude' in df.columns:
                gdf = gpd.GeoDataFrame(
                    df, 
                    geometry=gpd.points_from_xy(df.longitude, df.latitude),
                    crs="EPSG:4326"
                )
            
            # Sjekk etter potensielle geometrikolonner
            if gdf is None:
                geom_kolonner = [col for col in df.columns if 'geom' in col.lower() 
                              or 'coord' in col.lower() 
                              or 'point' in col.lower() 
                              or 'polygon' in col.lower()
                              or 'linestring' in col.lower()
                              or 'wkt' in col.lower()]
                
                for col in geom_kolonner:
                    try:
                        # Prøv å konvertere fra WKT-format hvis det er en tekstkolonne
                        if df[col].dtype == 'object':
                            from shapely import wkt
                            geom = df[col].apply(wkt.loads)
                            gdf = gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
                            break
                    except Exception:
                        pass
        
        elif filendelse in ['.geojson', '.json']:
            # Les GeoJSON-filen uten å spesifisere driver for å unngå advarsel
            gdf = gpd.read_file(kildefilsti)
            
        elif filendelse == '.shp':
            # Les shapefile
            gdf = gpd.read_file(kildefilsti)
            
        elif filendelse == '.gpkg':
            # Les GeoPackage
            gdf = gpd.read_file(kildefilsti, driver='GPKG')
            
        elif filendelse == '.csv':
            # Prøv å lese CSV med lat/long kolonner
            df = pd.read_csv(kildefilsti)
            
            # Sjekk for vanlige lat/long kolonnenavn
            lat_kolonner = ['latitude', 'lat', 'y', 'breddegrad']
            lon_kolonner = ['longitude', 'long', 'lon', 'x', 'lengdegrad']
            
            lat_col = next((col for col in lat_kolonner if col in df.columns), None)
            lon_col = next((col for col in lon_kolonner if col in df.columns), None)
            
            if lat_col and lon_col:
                gdf = gpd.GeoDataFrame(
                    df, 
                    geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
                    crs="EPSG:4326"
                )
        
        # Hvis vi har en gyldig geodataframe, lagre som GeoParquet
        if gdf is not None:
            gdf.to_parquet(målfilsti)
            return målfilsti
        
        return False
        
    except Exception as e:
        print(f"Feil ved konvertering av {kildefilsti}: {e}")
        return False

In [5]:
def finn_nye_filer(rå_mappesti, prosessert_mappesti, konverteringslogg_sti=None):
    """Finner filer som ikke har blitt konvertert ennå"""
    # Støttede filformater
    støttede_formater = ['.parquet', '.geojson', '.json', '.shp', '.gpkg', '.csv']
    
    # Hent alle allerede konverterte filer
    konverterte_filer = set()
    if os.path.exists(prosessert_mappesti):
        for root, _, filer in os.walk(prosessert_mappesti):
            for fil in filer:
                if fil.endswith('.parquet'):
                    base_navn = os.path.splitext(fil)[0]
                    konverterte_filer.add(base_navn)
    
    # Last inn konverteringsloggen hvis den finnes
    konverterte_logger = {}
    if konverteringslogg_sti and os.path.exists(konverteringslogg_sti):
        try:
            with open(konverteringslogg_sti, 'r') as f:
                konverterte_logger = json.load(f)
        except Exception:
            pass
    
    # Finn alle filer som må konverteres
    nye_filer = []
    for root, _, filer in os.walk(rå_mappesti):
        for fil in filer:
            filsti = os.path.join(root, fil)
            filendelse = os.path.splitext(fil)[1].lower()
            base_navn = os.path.splitext(fil)[0]
            
            # Sjekk om filen har et støttet format
            if filendelse in støttede_formater:
                # Parquet-filer trenger en ekstra sjekk
                if filendelse == '.parquet':
                    # Ikke ta med filer som allerede er GeoParquet
                    if er_geoparquet(filsti):
                        continue
                    # Ikke ta med filer som allerede har _geo suffix
                    if base_navn.endswith('_geo'):
                        continue
                
                # Sjekk om filen allerede er konvertert
                if base_navn not in konverterte_filer:
                    # Sjekk om filen har endret seg siden sist konvertering
                    if base_navn in konverterte_logger:
                        sist_endret_tid = os.path.getmtime(filsti)
                        sist_konvertert_tid = konverterte_logger[base_navn].get('tidspunkt', 0)
                        if sist_endret_tid <= sist_konvertert_tid:
                            continue
                    
                    nye_filer.append(filsti)
    
    return nye_filer

In [6]:
def opprett_mappestruktur(rotmappe):
    """Oppretter den anbefalte mappestrukturen"""
    mapper = [
        os.path.join(rotmappe, 'raw'),
        os.path.join(rotmappe, 'processed'),
        os.path.join(rotmappe, 'archive')
    ]
    
    for mappe in mapper:
        os.makedirs(mappe, exist_ok=True)
    
    return {
        'raw': mapper[0],
        'processed': mapper[1],
        'archive': mapper[2]
    }

In [7]:
def oppdater_konverteringslogg(konverteringslogg_sti, fil, målfilsti):
    """Oppdaterer konverteringsloggen med ny filinfo"""
    konverterte_logger = {}
    
    # Last inn eksisterende logg hvis den finnes
    if os.path.exists(konverteringslogg_sti):
        try:
            with open(konverteringslogg_sti, 'r') as f:
                konverterte_logger = json.load(f)
        except Exception:
            pass
    
    # Legg til ny informasjon
    base_navn = os.path.splitext(os.path.basename(fil))[0]
    konverterte_logger[base_navn] = {
        'kildesti': fil,
        'målsti': målfilsti,
        'tidspunkt': datetime.datetime.now().timestamp(),
        'dato': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Lagre oppdatert logg
    with open(konverteringslogg_sti, 'w') as f:
        json.dump(konverterte_logger, f, indent=2)

In [8]:
def behandle_alle_filer_i_mappe(rotmappesti, opprett_struktur=True):
    """Behandler alle geografiske filer i angitt mappe og konverterer til GeoParquet."""
    # Sjekk om katalogen eksisterer
    if not os.path.exists(rotmappesti):
        print(f"FEIL: Katalogen '{rotmappesti}' eksisterer ikke.")
        return None
    
    mappestruktur = None
    rå_mappesti = rotmappesti
    prosessert_mappesti = rotmappesti
    
    # Opprett mappestruktur hvis ønsket
    if opprett_struktur:
        mappestruktur = opprett_mappestruktur(rotmappesti)
        rå_mappesti = mappestruktur['raw']
        prosessert_mappesti = mappestruktur['processed']
        print(f"Opprettet mappestruktur:\n- Raw: {rå_mappesti}\n- Processed: {prosessert_mappesti}")
    
    # Konverteringslogg-sti
    konverteringslogg_sti = os.path.join(rotmappesti, 'conversion_log.json')
    
    # Finn nye filer som trenger konvertering
    nye_filer = finn_nye_filer(rå_mappesti, prosessert_mappesti, konverteringslogg_sti)
    
    resultater = {
        "allerede_geoparquet": [],
        "konvertert": [],
        "feilet": [],
        "alle_filer": []
    }
    
    # Konverter filer
    for filsti in nye_filer:
        resultater["alle_filer"].append(filsti)
        
        # Konverter filen
        målfilsti = konverter_til_geoparquet(filsti, prosessert_mappesti)
        
        if målfilsti:
            resultater["konvertert"].append(filsti)
            oppdater_konverteringslogg(konverteringslogg_sti, filsti, målfilsti)
        else:
            resultater["feilet"].append(filsti)
    
    # Finn allerede eksisterende GeoParquet-filer
    for root, _, filer in os.walk(rå_mappesti):
        for fil in filer:
            if fil.endswith('.parquet'):
                filsti = os.path.join(root, fil)
                if er_geoparquet(filsti) and filsti not in resultater["alle_filer"]:
                    resultater["allerede_geoparquet"].append(filsti)
                    resultater["alle_filer"].append(filsti)
    
    # Oppsummering
    print("\n" + "="*50)
    print("OPPSUMMERING")
    print("="*50)
    print(f"Katalog: {rotmappesti}")
    if mappestruktur:
        print(f"Mappestruktur: Raw={rå_mappesti}, Processed={prosessert_mappesti}")
    print(f"Totalt antall geografiske filer funnet: {len(resultater['alle_filer'])}")
    print(f"Antall parquet-filer som allerede var GeoParquet: {len(resultater['allerede_geoparquet'])}")
    print(f"Antall filer konvertert til GeoParquet: {len(resultater['konvertert'])}")
    print(f"Antall filer som ikke kunne konverteres: {len(resultater['feilet'])}")
    
    if resultater["konvertert"]:
        print("\nKonverterte filer:")
        for fil in resultater["konvertert"]:
            print(f"- {fil}")
    
    if resultater["feilet"]:
        print("\nFiler som ikke kunne konverteres:")
        for fil in resultater["feilet"]:
            print(f"- {fil}")
    
    return resultater

In [11]:
# Angi rotmappen som skal behandles
resultater = behandle_alle_filer_i_mappe(ROTMAPPE, opprett_struktur=True)

Opprettet mappestruktur:
- Raw: Data\raw
- Processed: Data\processed

OPPSUMMERING
Katalog: Data
Mappestruktur: Raw=Data\raw, Processed=Data\processed
Totalt antall geografiske filer funnet: 0
Antall parquet-filer som allerede var GeoParquet: 0
Antall filer konvertert til GeoParquet: 0
Antall filer som ikke kunne konverteres: 0


## Filtering

In [18]:
# MÅ HENTE RIKTIG FIL!

# Les geoparquet-filen
fil_sti = f"{ROTMAPPE}/raw/hais_2025-01-01.snappy.parquet"
gdf = gpd.read_parquet(fil_sti)

# Filtrering basert på attributt/kolonne
if 'date_time_utc' in gdf.columns:
    filtrert_på_dato = gdf[gdf['date_time_utc'] >= '2025-01-01 12:00:00']
    print(f"Antall rader etter datofiltrering: {len(filtrert_på_dato)}")
    display(filtrert_på_dato.head())

# Filtrering basert på geometri (f.eks. et område)
område = box(5.0, 60.0, 11.0, 60.0)
innenfor_område = gdf[gdf.geometry.intersects(område)]
print(f"Antall rader innenfor det definerte området: {len(innenfor_område)}")
display(innenfor_område.head())

# Kombinert filtrering med flere kriterier
if 'attributt' in gdf.columns:
    kombinert_filter = gdf[(gdf.geometry.intersects(område)) & (gdf['attributt'] > 10)]
    print(f"Antall rader etter kombinert filtrering: {len(kombinert_filter)}")
    display(kombinert_filter.head())

FileNotFoundError: [WinError 2] Failed to open local file 'Data/raw/hais_2025-01-01.snappy.parquet'. Detail: [Windows error 2] Systemet finner ikke angitt fil.


## Partisjonerte geoparquet-filer

In [ ]:
# sett i funskjon?

# 1. Opprett en mappe for partisjonerte data
os.makedirs(PARTISJON_MAPPE_HOUR, exist_ok=True)

# 2. Lag en kopi for å unngå SettingWithCopyWarning
data_for_partisjonering = innenfor_område.copy()

# 3. Bruk .loc for å legge til time-kolonnen
data_for_partisjonering.loc[:, 'time'] = data_for_partisjonering['date_time_utc'].dt.hour

# 4. Partisjonering basert på time og ship_type
for time, gruppe_time in data_for_partisjonering.groupby('time'):
    time_mappe = os.path.join(PARTISJON_MAPPE_HOUR, f"time={time:02d}")
    os.makedirs(time_mappe, exist_ok=True)
    
    for ship_type, gruppe_final in gruppe_time.groupby('ship_type'):
        skip_mappe = os.path.join(time_mappe, f"ship_type={ship_type}")
        os.makedirs(skip_mappe, exist_ok=True)
        
        # Bruk en kopi av dataene
        data_å_lagre = gruppe_final.copy()
        
        # Fjern partisjoneringskolonnen før lagring
        fil_sti = os.path.join(skip_mappe, f"data.parquet")
        data_å_lagre.drop('time', axis=1).to_parquet(fil_sti)
        
        print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")

In [ ]:
# Dele opp i funskjoner

# Hente data for en spesifikk måned
ønsket_måned = "2025-01"
måned_sti = os.path.join(PARTISJON_MAPPE_HOUR, f"år_måned={ønsket_måned}")

if os.path.exists(måned_sti):
    print(f"Leser data for {ønsket_måned}...")
    # Finn alle geoparquet-filer i denne månedens mappe (inkludert undermapper)
    filer = glob.glob(os.path.join(måned_sti, "**/*.parquet"), recursive=True)
    # Les og kombiner alle filene
    dataframes = []
    for fil in filer:
        gdf = gpd.read_parquet(fil)
        dataframes.append(gdf)
    
    if dataframes:
        månedsdata = pd.concat(dataframes)
        print(f"Hentet {len(månedsdata)} rader for {ønsket_måned}")
        display(månedsdata.head())
    else:
        print(f"Ingen data funnet for {ønsket_måned}")
else:
    print(f"Ingen mappe funnet for {ønsket_måned}")

# Hente data for en spesifikk skipstype
ønsket_skipstype = 70 
skipstype_stier = glob.glob(os.path.join(PARTISJON_MAPPE_HOUR, f"**/ship_type={ønsket_skipstype}/*.parquet"), recursive=True)

if skipstype_stier:
    print(f"\nLeser data for skipstype {ønsket_skipstype}...")
    skipstype_dataframes = []
    for fil in skipstype_stier:
        gdf = gpd.read_parquet(fil)
        skipstype_dataframes.append(gdf)
    
    skipstype_data = pd.concat(skipstype_dataframes)
    print(f"Hentet {len(skipstype_data)} rader for skipstype {ønsket_skipstype}")
    display(skipstype_data.head())
else:
    print(f"Ingen data funnet for skipstype {ønsket_skipstype}")

In [ ]:
# 1. Opprett en mappe for partisjonerte data
os.makedirs(PARTISJON_MAPPE_MIN, exist_ok=True)

# 2. Lag en kopi for å unngå SettingWithCopyWarning
data_for_partisjonering = innenfor_område.copy()

# 3. Bruk .loc for å legge til 10-minutters intervall kolonne
# Dette gir intervaller 0-143 for hele dagen (144 intervaller på 10 minutter)
data_for_partisjonering.loc[:, 'minuttgruppe'] = (
    data_for_partisjonering['date_time_utc'].dt.hour * 6 + 
    data_for_partisjonering['date_time_utc'].dt.minute // 10
)

# 4. Partisjonering basert på minuttgruppe og ship_type
for minuttgruppe, gruppe_minutt in data_for_partisjonering.groupby('minuttgruppe'):
    # Konverter minuttgruppe til time og minutt for mappe-strukturen
    time = minuttgruppe // 6
    minutt = (minuttgruppe % 6) * 10
    
    # Lag en lesbar mappestruktur (time_minutt=HH_MM)
    minutt_mappe = os.path.join(PARTISJON_MAPPE_MIN, f"time_minutt={time:02d}_{minutt:02d}")
    os.makedirs(minutt_mappe, exist_ok=True)
    
    for ship_type, gruppe_final in gruppe_minutt.groupby('ship_type'):
        skip_mappe = os.path.join(minutt_mappe, f"ship_type={ship_type}")
        os.makedirs(skip_mappe, exist_ok=True)
        
        # Bruk en kopi av dataene
        data_å_lagre = gruppe_final.copy()
        
        # Fjern partisjoneringskolonnen før lagring
        fil_sti = os.path.join(skip_mappe, f"data.parquet")
        data_å_lagre.drop('minuttgruppe', axis=1).to_parquet(fil_sti)
        
        print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")

## Simulering av datastrømming

In [ ]:
# Simulert datastrømming med visualisering

# Finn alle minutt-mapper sortert kronologisk
alle_intervaller = sorted([d for d in os.listdir(PARTISJON_MAPPE_MIN) if d.startswith("time_minutt=")])

# Funksjon for å lage et kart med skipsbevegelser
def vis_skip_på_kart(data, tidspunkt):
    # Konverter til WGS84 koordinatsystem hvis nødvendig
    if data.crs and data.crs != "EPSG:4326":
        data = data.to_crs("EPSG:4326")
    
    # Beregn senterpunkt for kartet
    midpoint_lat = data.geometry.centroid.y.mean()
    midpoint_lon = data.geometry.centroid.x.mean()
    
    # Opprett et kart sentrert på dataenes midtpunkt
    m = folium.Map(location=[midpoint_lat, midpoint_lon], zoom_start=10)
    
    # Legg til en tittel
    tittel_html = f'''
    <h3 align="center" style="font-size:16px"><b>Skipsbevegelser: {tidspunkt}</b></h3>
    '''
    m.get_root().html.add_child(folium.Element(tittel_html))
    
    # Fargekoding basert på skipstype
    ship_type_colors = {
        30: 'blue',      # Fiskefartøy
        31: 'green',     # Slepebåt
        52: 'red',       # Passasjerskip
        60: 'purple',    # Passasjerskip
        70: 'orange',    # Lasteskip
        80: 'darkblue',  # Tankskip
        # Legg til flere skipstyper etter behov
    }
    
    # Legg til hvert skip på kartet
    for idx, row in data.iterrows():
        # Bestem farge basert på skipstype
        color = ship_type_colors.get(row['ship_type'], 'gray')
        
        # Hent koordinater
        coords = (row.geometry.y, row.geometry.x)
        
        # Lag popup-info
        popup_text = f"""
        <b>Skip:</b> {row['ship_name']}<br>
        <b>MMSI:</b> {row['mmsi']}<br>
        <b>Type:</b> {row['ship_type']}<br>
        <b>Hastighet:</b> {row['speed_over_ground']} knop<br>
        <b>Kurs:</b> {row['course_over_ground']}°<br>
        <b>Tidspunkt:</b> {row['date_time_utc']}
        """
        
        # Legg til markør med retningspil
        folium.Marker(
            coords,
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color=color, icon='ship', prefix='fa'),
            tooltip=f"{row['ship_name']} ({row['mmsi']})"
        ).add_to(m)
        
        # Tegn retningspil hvis kurs er tilgjengelig
        if not pd.isna(row['course_over_ground']):
            folium.RegularPolygonMarker(
                coords,
                fill_color=color,
                number_of_sides=3,
                radius=8,
                rotation=row['course_over_ground'],
                fill_opacity=0.6,
                color='black',
                weight=1
            ).add_to(m)
    
    # Legg til tegnforklaring
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 150px; height: 160px; 
                border:2px solid grey; z-index:9999; font-size:12px;
                background-color: white; padding: 10px;
                border-radius: 5px;">
    <b>Skipstyper:</b><br>
    '''
    
    for ship_type, color in ship_type_colors.items():
        legend_html += f'<i class="fa fa-ship" style="color:{color}"></i> Type {ship_type}<br>'
    
    legend_html += '</div>'
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Returner kartet
    return m

# Simuler datastrømming med visualisering
print("\nSimulerer datastrømming med visualisering...")
for intervall_dir in alle_intervaller:
    # Hent time og minutt fra mappenavnet
    match = re.search(r"time_minutt=(\d{2})_(\d{2})", intervall_dir)
    if match:
        time, minutt = match.groups()
        tidspunkt = f"{time}:{minutt}"
        
        intervall_sti = os.path.join(PARTISJON_MAPPE_MIN, intervall_dir)
        print(f"\nProsesserer data for {tidspunkt}...")
        
        # Finn alle geoparquet-filer for dette intervallet
        filer = glob.glob(os.path.join(intervall_sti, "**/*.parquet"), recursive=True)
        
        # Les og bearbeid hver fil
        intervall_data = []
        for fil in filer:
            gdf = gpd.read_parquet(fil)
            intervall_data.append(gdf)
            
        if intervall_data:
            samlet_data = pd.concat(intervall_data)
            print(f"Kl. {tidspunkt}: Lastet {len(samlet_data)} rader fra {len(filer)} filer")
            
            # Vis statistikker
            print(f"  Gjennomsnittlig hastighet (SOG): {samlet_data['speed_over_ground'].mean():.2f}")
            print(f"  Antall unike skip: {samlet_data['mmsi'].nunique()}")
            
            # Opprett og vis kart
            kart = vis_skip_på_kart(samlet_data, f"Kl. {tidspunkt}")
            
            # Lagre kartet til en HTML-fil
            # kart_filnavn = f"kart_{time}_{minutt}.html"
            # kart.save(kart_filnavn)
            # print(f"  Kart lagret som {kart_filnavn}")
            
            display(kart)
            
        else:
            print(f"Ingen data funnet for kl. {tidspunkt}")
        
        # Simuler tid mellom intervaller
        time_module.sleep(0.5)
        
print("\nDatastrømming komplett!")